In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing  import StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet,SGDRegressor,BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.metrics import r2_score,root_mean_squared_error
import joblib




import warnings
warnings.filterwarnings("ignore")

In [147]:
data = pd.read_csv('/Users/saptarshipal/DATA/boston_house_pricing_regression/data/Delhi_v2.csv')
data = pd.DataFrame(data)


In [148]:
data = data.drop(columns = ['Unnamed: 0','Address','Landmarks','desc'])
data.head()

,price,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,Status,neworold,parking,Furnished_status,Lift,type_of_building,Price_sqft
0,5600000.0,1350.0,28.608850,77.460560,3.0,3.0,NaN,Under Construction,New Property,NaN,NaN,2.0,Flat,4148.148148
1,8800000.0,1490.0,28.374236,76.952416,3.0,3.0,NaN,Ready to Move,New Property,NaN,Semi-Furnished,2.0,Flat,5906.040268
2,16500000.0,2385.0,28.645769,77.385110,4.0,5.0,NaN,Ready to Move,New Property,1.0,Unfurnished,NaN,Flat,6918.238994
3,3810000.0,1050.0,28.566914,77.436434,2.0,2.0,3.0,NaN,New Property,1.0,Unfurnished,2.0,Flat,3628.571429
4,6200000.0,1350.0,28.520732,77.356491,2.0,2.0,3.0,Ready to Move,Resale,1.0,NaN,3.0,Flat,4592.592593


In [149]:
data.columns

Index(['price', 'area', 'latitude', 'longitude', 'Bedrooms', 'Bathrooms',
       'Balcony', 'Status', 'neworold', 'parking', 'Furnished_status', 'Lift',
       'type_of_building', 'Price_sqft'],
      dtype='object')

In [150]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7738 entries, 0 to 7737
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             7738 non-null   float64
 1   area              7738 non-null   float64
 2   latitude          7738 non-null   float64
 3   longitude         7738 non-null   float64
 4   Bedrooms          7738 non-null   float64
 5   Bathrooms         7738 non-null   float64
 6   Balcony           5166 non-null   float64
 7   Status            7164 non-null   object 
 8   neworold          7738 non-null   object 
 9   parking           2612 non-null   float64
 10  Furnished_status  4124 non-null   object 
 11  Lift              1733 non-null   float64
 12  type_of_building  7738 non-null   object 
 13  Price_sqft        7738 non-null   float64
dtypes: float64(10), object(4)
memory usage: 846.5+ KB


In [151]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
price,7738.0,8.320635e+06,7.223197e+06,1.700000e+06,4.200000e+06,6.000000e+06,9.500000e+06,8.500000e+07
area,7738.0,1.409507e+03,7.189296e+02,5.010000e+02,9.900000e+02,1.250000e+03,1.650000e+03,9.500000e+03
latitude,7738.0,2.855209e+01,1.074200e-01,2.824002e+01,2.845554e+01,2.857464e+01,2.864252e+01,2.879975e+01
longitude,7738.0,7.727348e+01,1.806062e-01,7.688410e+01,7.707859e+01,7.734532e+01,7.742105e+01,7.768803e+01
Bedrooms,7738.0,2.708193e+00,8.770258e-01,2.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00,1.000000e+01
Bathrooms,7738.0,2.501163e+00,8.670501e-01,2.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.000000e+01
Balcony,5166.0,2.426442e+00,1.083677e+00,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.000000e+01
parking,2612.0,7.673047e+00,6.041751e+01,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.406000e+03
Lift,1733.0,1.829198e+00,1.192607e+00,0.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,1.000000e+01
Price_sqft,7738.0,5.543660e+03,2.408659e+03,2.100000e+03,3.950987e+03,4.972674e+03,6.350639e+03,4.437870e+04


# Imputing

In [152]:
imputer1 = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imputer2 = SimpleImputer(missing_values=pd.NA,strategy='most_frequent')

data['Balcony'] = imputer1.fit_transform(data[['Balcony']] )
data['Status'] = imputer2.fit_transform(data[['Status']])
data['parking'] = imputer1.fit_transform(data[['parking']])
data['Furnished_status'] = imputer2.fit_transform(data[['Furnished_status']])
data['Lift'] = imputer1.fit_transform(data[['Lift']])

ValueError: 2

In [ ]:
data[data['Status'].isnull()]

,price,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,Status,neworold,parking,Furnished_status,Lift,type_of_building,Price_sqft
3,3810000.0,1050.0,28.566914,77.436434,2.0,2.0,3.0,NaN,New Property,1.0,Unfurnished,2.0,Flat,3628.571429
19,14000000.0,1845.0,28.546930,77.354387,3.0,3.0,3.0,NaN,Resale,2.0,Semi-Furnished,2.0,Flat,7588.075881
34,15000000.0,1960.0,28.390730,77.057823,3.0,3.0,2.0,NaN,Resale,1.0,Semi-Furnished,2.0,Flat,7653.061224
67,5500000.0,1350.0,28.649348,77.369626,2.0,2.0,2.0,NaN,Resale,1.0,Furnished,2.0,Flat,4074.074074
78,21000000.0,2800.0,28.465591,77.511403,4.0,5.0,4.0,NaN,Resale,1.0,Unfurnished,2.0,Flat,7500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,8000000.0,1712.0,28.414052,76.914303,4.0,3.0,3.0,NaN,Resale,1.0,Unfurnished,2.0,Flat,4672.897196
7714,4500000.0,995.0,28.477342,77.552934,3.0,3.0,2.0,NaN,Resale,1.0,Unfurnished,2.0,Flat,4522.613065
7719,9500000.0,1850.0,28.441405,77.476994,3.0,3.0,2.0,NaN,Resale,1.0,Semi-Furnished,2.0,Flat,5135.135135
7720,13000000.0,1692.0,28.476311,76.971611,3.0,3.0,3.0,NaN,Resale,1.0,Semi-Furnished,2.0,Individual House,7683.215130


In [ ]:
data.isnull().sum()

price                  0
area                   0
latitude               0
longitude              0
Bedrooms               0
Bathrooms              0
Balcony                0
Status               574
neworold               0
parking                0
Furnished_status    3614
Lift                   0
type_of_building       0
Price_sqft             0
dtype: int64

# Encoding

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7738 entries, 0 to 7737
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             7738 non-null   float64
 1   area              7738 non-null   float64
 2   latitude          7738 non-null   float64
 3   longitude         7738 non-null   float64
 4   Bedrooms          7738 non-null   float64
 5   Bathrooms         7738 non-null   float64
 6   Balcony           7738 non-null   object 
 7   Status            7738 non-null   object 
 8   neworold          7738 non-null   object 
 9   parking           7738 non-null   object 
 10  Furnished_status  7738 non-null   object 
 11  Lift              7738 non-null   object 
 12  type_of_building  7738 non-null   object 
 13  Price_sqft        7738 non-null   float64
dtypes: float64(7), object(7)
memory usage: 846.5+ KB


In [ ]:
encoder1 = LabelEncoder()
columns1 = ['Status','neworold','Furnished_status','type_of_building']

data['Status'] = encoder1.fit_transform(data['Status'])

data.head()

,price,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,Status,neworold,parking,Furnished_status,Lift,type_of_building,Price_sqft
0,5600000.0,1350.0,28.608850,77.460560,3.0,3.0,SimpleImputer(strategy='most_frequent'),0,New Property,SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),Flat,4148.148148
1,8800000.0,1490.0,28.374236,76.952416,3.0,3.0,SimpleImputer(strategy='most_frequent'),0,New Property,SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),Flat,5906.040268
2,16500000.0,2385.0,28.645769,77.385110,4.0,5.0,SimpleImputer(strategy='most_frequent'),0,New Property,SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),Flat,6918.238994
3,3810000.0,1050.0,28.566914,77.436434,2.0,2.0,SimpleImputer(strategy='most_frequent'),0,New Property,SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),Flat,3628.571429
4,6200000.0,1350.0,28.520732,77.356491,2.0,2.0,SimpleImputer(strategy='most_frequent'),0,Resale,SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),SimpleImputer(strategy='most_frequent'),Flat,4592.592593
